# Run End-to-end Group Chat

This Jupyter Notebook demonstrates the workflow for creating and managing a group chat using Semantic Kernel and custom modules. The notebook integrates various Azure components, such as Key Vault, Blob Storage, and OpenAI services, to facilitate secure and efficient communication. Key functionalities include:

- Setting up environment variables and configurations for Azure services.
- Initializing and managing chat contexts and data access layers.
- Creating a group chat using custom modules and Azure credentials.
- Processing chat messages asynchronously and invoking responses.
- Exploring chat history and analyzing message content.

This notebook serves as a comprehensive guide for implementing a scalable and secure group chat solution leveraging Azure's cloud capabilities.

In [ ]:
import os
import sys

CUR_DIR = os.path.abspath("")
ROOT_DIR = os.path.dirname(CUR_DIR)
SRC_DIR = os.path.join(ROOT_DIR, 'src')

# Load python modules from the src directory
sys.path.append(SRC_DIR)
print(f"Added {SRC_DIR} to sys.path")

In [ ]:
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(os.path.join(SRC_DIR, '.env'))

In [ ]:
import logging
from config import setup_logging

setup_logging(log_level=logging.INFO)

In [ ]:
from app import create_app_context
from data_models.chat_context import ChatContext


app_ctx = create_app_context()
chat_ctx = ChatContext("notebook-chat")

In [ ]:
import group_chat


chat, context = group_chat.create_group_chat(app_ctx, chat_ctx)

In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole


async def process_chat(chat: AgentGroupChat, user_message: str) -> None:
    """Process group chat with the given user message."""
    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_message))
    chat.is_complete = False

    async for response in chat.invoke():
        print(f"# {response.role} - {response.name or '*'}: '{response.content}'")
        if chat.is_complete:
            break


def print_chat_history(chat: AgentGroupChat) -> None:
    """Print the chat history."""
    for message in chat.history:
        print(f"{message.name}: {message.content}")

In [ ]:
await process_chat(chat, "Orchestrator: Prepare tumor board for Patient ID: patient_4")

In [ ]:
await process_chat(chat, "Orchestrator: proceed")

In [ ]:
# Uncomment the following line to print the chat history
# print_chat_history(chat)

In [ ]:
# Uncomment the following line to proceed with the chat
# await process_chat(chat, "Orchestrator: this information will be obtained by PatientHistory")